In [ ]:
from sentence_transformers import SentenceTransformer
import torch

# 1. Inicialització (fa un sol cop al començament del script)
text_model = SentenceTransformer('intfloat/multilingual-e5-base')
text_model.eval()  # Mode inferència

# 2. Funció per processar cada producte
def create_text_embedding(product_row):
    """
    product_row: pandas Series amb els atributs del producte
    """
    # Construïm la cadena com al teu exemple original
    attributes_string = f"{product_row['family']} {product_row['silhouette_type']} {product_row['waist_type']} para {product_row['moment']}"
    
    # CRÍTIC: Afegir el prefix "query:" per a optimitzar la cerca
    # Això és com el model espera les entrades
    formatted_text = f"query: {attributes_string}"
    
    # Generar embedding (tensor de PyTorch per facilitat)
    with torch.no_grad():
        embedding = text_model.encode(
            [formatted_text], 
            convert_to_tensor=True, 
            normalize_embeddings=True  # Normalitza a norma 1 (crucial per cosine similarity)
        )
    
    # Output shape: (1, 768)
    return embedding.squeeze(0)  # Retorna tensor de 768 dimensions

# 3. Exemple d'ús al teu pipeline
def process_dataset(df):
    """
    df: DataFrame amb les columnes 'family', 'silhouette_type', 'waist_type', 'moment'
    """
    embeddings_list = []
    
    for idx, row in df.iterrows():
        emb = create_text_embedding(row)
        embeddings_list.append(emb.cpu().numpy())  # Convertir a numpy per pandas
    
    # Afegir al DataFrame
    df['text_embedding'] = embeddings_list
    
    # Concatenar amb image_embedding (que ja tens)
    # Suposant que image_embedding és tensor de (N, 512)
    df['multimodal_embedding'] = df.apply(
        lambda row: torch.cat([
            torch.tensor(row['image_embedding']), 
            torch.tensor(row['text_embedding'])
        ], dim=0).numpy(),
        axis=1
    )
    
    return df